In [1]:
import os
import glob
import numpy as np
import pandas as pd
import ast 
import statistics
from datetime import datetime 

def get_prefix_from_path(file_path):
    if not file_path:
        return 'default'

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parts = base_name.split('_')
    
    if len(parts) >= 2:
        return '_'.join(parts[:2])
    else:
        return base_name

def issue(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,prefix):

    df=pd.read_csv('issue.csv')
    df=pd.DataFrame(df)

    data_comment = pd.read_csv(input_file_path1)['指标值'] if input_file_path1 else np.zeros(300, dtype=int)
    data_dura = pd.read_csv(input_file_path2)['指标值']  if input_file_path2 else np.zeros(300, dtype=int)
    data_respontime = pd.read_csv(input_file_path3)['指标值'] if input_file_path3 else np.zeros(300, dtype=int)
    data_new = pd.read_csv(input_file_path4)['指标值'] if input_file_path4 else np.zeros(300, dtype=int)
    data_closed = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)
    
    if input_file_path2:

        data_duration=ast.literal_eval(data_dura[0])

        # 过滤并转换时间和值
        times_floats = []
        for key, value in data_duration.items():
            if isinstance(key, str) and len(key) == 7 and key[4] == '-':  # 简单检查是否为YYYY-MM格式
                try:
                    # 将字符串时间转换为datetime对象
                    date_obj = datetime.strptime(key, '%Y-%m')
                    # 尝试将值转换为float
                    float_value = float(value)
                    times_floats.append((date_obj, float_value))
                except (ValueError, TypeError):
                    # 如果转换失败，跳过该项
                    continue
        # 按照日期排序
        times_floats.sort(key=lambda x: x[0])
        # 分离时间和值到两个独立的列表中
        dates_duration, values_duration = zip(*times_floats) if times_floats else ([], [])
       
        s_duration=[]
        
        for i in range(len(values_duration)):
            if values_duration[i]>10 :
                s_duration.append(10)
            else:
                s_duration.append(values_duration[i])
        avg4=np.mean(s_duration)
    else :
        avg4=0    
       
    if input_file_path3:
        data_response=ast.literal_eval(data_respontime[0]) 
        times_floats = []
        for key, value in data_response.items():
            if isinstance(key, str) and len(key) == 7 and key[4] == '-':  # 简单检查是否为YYYY-MM格式
                try:
                    # 将字符串时间转换为datetime对象
                    date_obj = datetime.strptime(key, '%Y-%m')
                    # 尝试将值转换为float
                    float_value = float(value)
                    times_floats.append((date_obj, float_value))
                except (ValueError, TypeError):
                    # 如果转换失败，跳过该项
                    continue
        # 按照日期排序
        times_floats.sort(key=lambda x: x[0])
        # 分离时间和值到两个独立的列表中
        dates_response, values_response = zip(*times_floats) if times_floats else ([], [])
        s_response=[]
        for i in range(len(values_response)):
                if values_response[i]>20 :
                    s_response.append(20)
                else:
                    s_response.append(values_response[i]) 
        avg3=np.mean(s_response)
        avg3=avg3/2
    else:
        avg3=0

    s_closed=[]
    s_new=[]
   
    for i in range(len(data_comment)):
        if data_comment[i]!=0:
            s_closed.append(data_closed[i]/data_comment[i])
            s_new.append(data_new[i]/data_comment[i])
        else:
            s_closed.append(0)
            s_new.append(0)

    avg1=np.mean(s_closed)
    avg2=np.mean(s_new)

    final=avg1*10/6+avg2*10/3+avg3/4+avg4/4

    # 获取仓库名
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final

    # 保存更改回 CSV 文件
    df.to_csv('issue.csv', index=False)

def get_prefix_and_rest(filename):

    base_name = os.path.basename(filename)
    # 分离文件名和扩展名
    name_part, ext_part = os.path.splitext(base_name) 
    # 按下划线分割文件名部分，并确保至少有两个'_'
    parts = name_part.split('_')

    if len(parts) > 1:
        # 获取前两个'_'之间的部分作为前缀
        prefix = '_'.join(parts[:2])
        # 剩余部分加上原始扩展名作为后缀
        rest = '_'.join(parts[2:]) + ext_part if len(parts) > 2 else ext_part
    else:
        # 如果没有足够的'_'，则返回整个文件名作为前缀，空字符串作为后缀
        prefix = name_part
        rest = ext_part
    
    return prefix, rest

def process_files_with_same_prefix(base_dir):
    # 定义需要查找的文件后缀
    file_suffixes = [
        'issue_comments.csv','issue_resolution_duration.csv','issue_response_time.csv',
        'issues_new.csv','issues_closed.csv'
    ]
    
    # 获取所有csv文件
    all_files = glob.glob(os.path.join(base_dir, '*.csv'))
    # 创建一个字典来存储每个前缀对应的文件路径列表
    files_dict = {}
    
    for file in all_files:
        
        # 获取仓库和后缀的部分作为前缀
        prefix, suffix = get_prefix_and_rest(os.path.basename(file))
        
        key = next((s for s in file_suffixes if s==suffix), None)
        if key:  # prefix是仓库名，key是文件后缀匹配，file是路径
            if prefix not in files_dict:
                files_dict[prefix] = {}
            files_dict[prefix][key] = file
    # 遍历字典并调用activity函数
    for prefix, paths in files_dict.items():
        # 调用 activity 函数，传递对应前缀的所有文件路径
        input_file_path1=paths.get('issue_comments.csv')
        input_file_path2=paths.get('issue_resolution_duration.csv')
        input_file_path3=paths.get('issue_response_time.csv')
        input_file_path4=paths.get('issues_new.csv')
        input_file_path5=paths.get('issues_closed.csv')
        issue(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,prefix)

# 假设 base_dir 是你的文件夹路径
base_dir='./最终数据处理'
process_files_with_same_prefix(base_dir)